## Dependencies

In [1]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

In [2]:
# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import tensorflow as tf

In [3]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.regularizers import l2


In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,                   
                                   validation_split=0.2)

In [6]:
train_directory = 'Resources/Alzheimers_Dataset/train'
test_directory = 'Resources/Alzheimers_Dataset/test'

In [7]:
train_generator = train_datagen.flow_from_directory(
train_directory, 
target_size=(256,256), 
color_mode='rgb', 
batch_size=25, 
class_mode='categorical', 
subset='training',
shuffle=True,
seed=42
)
validation_generator = train_datagen.flow_from_directory(
train_directory,
target_size=(256,256), 
color_mode='rgb', 
batch_size=25, 
class_mode='categorical', 
subset='validation', 
shuffle=False
)

Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.


In [8]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
test_directory,
target_size=(256,256), 
color_mode='rgb', 
batch_size=25, 
class_mode='categorical', 
shuffle=False
)

Found 1279 images belonging to 4 classes.


Source for code: https://medium.com/@ericdnbn/how-to-import-images-into-a-jupyter-notebook-using-keras-in-3-steps-2b100b18238c

In [9]:
model = Sequential()
model.add(Conv2D(256, kernel_size=(3,3), activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, kernel_size=(5,5), activation='relu', kernel_regularizer=l2(l2=0.001)))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(16, activation='relu', kernel_regularizer=l2(l2=0.001)))
model.add(Dense(4, activation='softmax'))
            
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(train_generator,
          steps_per_epoch=40,
          epochs=50,
          validation_data=(validation_generator),
          validation_steps=10)


Epoch 1/50

40/40 [==============================] - 294s 7s/step - loss: 1.7967 - accuracy: 0.5010 - val_loss: 1.4740 - val_accuracy: 0.3880
Epoch 2/50
40/40 [==============================] - 281s 7s/step - loss: 1.2322 - accuracy: 0.4790 - val_loss: 1.8734 - val_accuracy: 0.3880
Epoch 3/50
40/40 [==============================] - 264s 7s/step - loss: 1.0819 - accuracy: 0.5050 - val_loss: 1.9446 - val_accuracy: 0.3800
Epoch 4/50
40/40 [==============================] - 263s 7s/step - loss: 1.0278 - accuracy: 0.5460 - val_loss: 2.1059 - val_accuracy: 0.0360
Epoch 5/50
40/40 [==============================] - 264s 7s/step - loss: 1.0692 - accuracy: 0.5080 - val_loss: 1.5585 - val_accuracy: 0.1440
Epoch 6/50
40/40 [==============================] - 264s 7s/step - loss: 0.9694 - accuracy: 0.5870 - val_loss: 2.0878 - val_accuracy: 0.2960
Epoch 7/50
40/40 [==============================] - 261s 7s/step - loss: 0.9881 - accuracy: 0.5800 - val_loss: 1.7324 - val_accuracy: 0.3240
Epoch 8/50


In [ ]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

In [ ]:

plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,50])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])

In [ ]:
val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)
print("Validation batch shape:", val_image_batch.shape)

In [ ]:
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

In [ ]:
tf_model_predictions = model.predict(val_image_batch)
print("Prediction results shape:", tf_model_predictions.shape)

In [ ]:
predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]
print(predicted_labels)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range((len(predicted_labels)-2)):
  plt.subplot(6,5,n+1)
  plt.imshow(val_image_batch[n])
  color = "green" if predicted_ids[n] == true_label_ids[n] else "red"
  plt.title(predicted_labels[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")